# Age and Biomass

This script outputs the satellite-based rasters into the Google Earth Engine Cloud.

Inputs:




In [2]:
import ee
import geemap
from utils import *
initialize()

config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder
last_year = config.last_year

mapbiomas, lulc = desired_lulc()


## Biomass - Export secondary

Biomass data is in hectares, but mapbiomas data is 30m resolution.

To deal with edge pixels, we aggregate biomass values to 30m by using the mean (so there is a buffer for land use pixels caught in between two biomass values).

## Remove isolated pixels

In the map, there were isolated pixels, often around the edges of forest patches. These would likely be due to misclassification, or follow different behaviors due to edge effects.

To avoid this issue, a kernel is applied here to include only secondary forest patches that are mostly surrounded by other secondary forest pixels.

In [17]:
def export_secondary(age, name):

    # # select the data points that match their classification as young or old by IPCC
    # filter_ages = ee.ImageCollection("NASA/ORNL/global_forest_classification_2020/V1").first().select("classification")
    # filter_young_secondary = filter_ages.eq(2)
    # filter_old_secondary = filter_ages.eq(3)
    # young_secondary = age.lte(20).updateMask(filter_young_secondary).unmask(0)
    # old_secondary = age.gt(20).updateMask(filter_old_secondary).unmask(0)
    # ipcc_mask = young_secondary.add(old_secondary)

    # # ----------------  Select only patches of at least one hectare  ----------------
    # # Identify all connected components (patches) in the image
    # age_patches = age.connectedComponents(ee.Kernel.plus(1), 256)
    # # Calculate the area of each patch
    # patchAreas = age_patches.select("labels").selfMask().addBands(ee.Image.pixelArea()).reduceConnectedComponents(ee.Reducer.sum(), 'labels', 256)
    # # Select only patches of at least one hectare
    # largePatches = patchAreas.gte(10000).selfMask()

    # ----------------  Exclude edge pixels ----------------
    # convert non-forest pixels from NA to zero
    # check what is the most frequent value within each hectare - if it's zero, it means the pixel is surrounded by non-forest cover
    exclude_edge_mask = age.unmask(0).focalMode(kernelType = "circle", radius = 100, units = "meters").gt(0).selfMask()

    distance_to_border_mask = ee.Image(f"{data_folder}/distance_to_border_mask")

    age = age.updateMask(exclude_edge_mask.And(distance_to_border_mask)).rename("age")

    export_image(age, name, region = roi, scale = 30)



In [ ]:


age = ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_secondary_vegetation_age_v1").select("secondary_vegetation_age_2020")


# age2 = age.updateMask(exclude_edge_mask.And(distance_to_border_mask)).rename("age")

edge_detec = age.unmask(-1).zeroCrossing()

distance_to_secondary = edge_detec.fastDistanceTransform(100, 'pixels').sqrt() \
    .multiply(ee.Image.pixelArea().sqrt()).toInt32().rename("distance_to_secondary")


export_image(distance_to_secondary, "distance_to_secondary", region = roi, scale = 30)


## Export EU TMF data for comparison with MapBiomas

In [15]:
# Load the image collections
transition = ee.ImageCollection('projects/JRC/TMF/v1_2023/TransitionMap_Subtypes').mosaic().clip(roi)
annual_changes = ee.ImageCollection('projects/JRC/TMF/v1_2023/AnnualChanges').mosaic().clip(roi)

# Define regrowth and degraded conditions
regrowth = transition.gte(31).And(transition.lte(33))

# Initialize AgeRegrowth and AgeDegraded
tmf = ee.Image.constant(0)

# Calculate AgeRegrowth
for i in range(1990, last_year):
    year = 'Dec' + str(i)
    annual_changes_year = annual_changes.select(year)
    condition = annual_changes_year.eq(4).And(regrowth) # were regrowing then AND are regrowing now
    tmf = tmf.add(condition.eq(1))

tmf = tmf.selfMask().rename(f"tmf_{last_year}")


In [18]:
export_secondary(mapbiomas, f"mapbiomas_{last_year}_full")
# export_secondary(tmf, f"tmf_{last_year}")

In [49]:
tmf = ee.Image(f"{data_folder}/tmf_{last_year}_1").rename("tmf")

ESA_CCI = ee.ImageCollection("projects/sat-io/open-datasets/ESA/ESA_CCI_AGB").filterDate('2019-01-01','2021-01-01').first().select("AGB").rename("biomass")

# ESA_CCI_resampled = ESA_CCI.reduceResolution(
#         reducer= ee.Reducer.mean(),
#     ).reproject(
#         crs=tmf.projection(),
#         scale=tmf.projection().nominalScale()
#     )

# tmf_mask = tmf.gt(0).selfMask().rename("tmf_mask")

tmf_ESA = ESA_CCI.addBands(tmf)#.addBands(tmf_mask)

# map = geemap.Map()
# map.addLayer(ESA_CCI_resampled, {}, "ESA_CCI_resampled")
# map.addLayer(tmf_ESA, {}, "tmf_ESA")
# map

tmf_ESA_fc = tmf_ESA.stratifiedSample(numPoints = 100,
                                      classBand = "tmf",
                                      dropNulls = True)

task = ee.batch.Export.table.toDrive(collection = tmf_ESA_fc, fileFormat="CSV")
task.start()